## Notebook with SPARQL Kernel

To run this notebook, install the [SPARQL kernel](https://github.com/paulovn/sparql-kernel):
```
pip install sparqlkernel
jupyter sparqlkernel install
```

To use Graphviz to draw graphs, you should also install Pydot and Graphviz:
```
conda install pydot graphviz
```
Set the path to the dot script in your PATH variable. For Miniconda users on Windows, the path would be like `C:\Users\user\Miniconda3\Library\bin\graphviz`

In [89]:
%endpoint https://data-issa.cirad.fr/sparql
%display table
%show 20

%prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
%prefix owl:    <http://www.w3.org/2002/07/owl#>
%prefix xsd:    <http://www.w3.org/2001/XMLSchema#>

%prefix agrovoc: <http://aims.fao.org/aos/agrovoc/>
%prefix dce:    <http://purl.org/dc/elements/1.1/>
%prefix dct:    <http://purl.org/dc/terms/>
%prefix issa:   <http://data-issa.cirad.fr/>
%prefix issapr: <http://data-issa.cirad.fr/property/>
%prefix oa:     <http://www.w3.org/ns/oa#>
%prefix prov:   <http://www.w3.org/ns/prov#>
%prefix schema: <http://schema.org/>
%prefix skos:   <http://www.w3.org/2004/02/skos/core#> 
%prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>

%prefix gn:     <http://www.geonames.org/ontology#>
%prefix geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
%prefix spatial: <http://jena.apache.org/spatial#>
%prefix units:  <http://www.opengis.net/def/uom/OGC/1.0/>

Endpoint set to: https://data-issa.cirad.fr/sparql
Display: table
Result maximum size: 20
Prefix set: rdfs: = <http://www.w3.org/2000/01/rdf-schema#>
Prefix set: owl: = <http://www.w3.org/2002/07/owl#>
Prefix set: xsd: = <http://www.w3.org/2001/XMLSchema#>
Prefix set: agrovoc: = <http://aims.fao.org/aos/agrovoc/>
Prefix set: dce: = <http://purl.org/dc/elements/1.1/>
Prefix set: dct: = <http://purl.org/dc/terms/>
Prefix set: issa: = <http://data-issa.cirad.fr/>
Prefix set: issapr: = <http://data-issa.cirad.fr/property/>
Prefix set: oa: = <http://www.w3.org/ns/oa#>
Prefix set: prov: = <http://www.w3.org/ns/prov#>
Prefix set: schema: = <http://schema.org/>
Prefix set: skos: = <http://www.w3.org/2004/02/skos/core#>
Prefix set: skosxl: = <http://www.w3.org/2008/05/skos-xl#>
Prefix set: gn: = <http://www.geonames.org/ontology#>
Prefix set: geo: = <http://www.w3.org/2003/01/geo/wgs84_pos#>
Prefix set: spatial: = <http://jena.apache.org/spatial#>
Prefix set: units: = <http://www.opengis.net/def/uom/OGC/1.0/>

____

# Geographic named entities

### 1. Get the geographic NEs of a specific article

Look for the geographic named entities of article http://data-issa.cirad.fr/article/455916, and retrieve their details from GeoNames (stored locally in named graph http://geonames.org/graph).

The alternate names of the entity are concatenated in variable ?alternateNames.

In [138]:
SELECT ?geoEntity ?name ?latitude ?longitude ?altitude ?officialName (group_concat(distinct ?alternateName ; separator=", ") as ?alternateNames) ?nameParentCountry
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
WHERE {
    []  a issa:GeographicAnnotation;
        schema:about <http://data-issa.cirad.fr/article/455916>;
        oa:hasBody ?geoEntity.
    
    ?geoEntity
        gn:name   ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
        OPTIONAL {
            ?geoEntity gn:officialName ?officialName
            FILTER langMatches(lang(?officialName), "en")
        }.
        OPTIONAL {
            ?geoEntity gn:alternateName ?alternateName.
            FILTER langMatches(lang(?alternateName), "en")
        }.
        OPTIONAL { ?geoEntity geo:alt ?altitude }.
        OPTIONAL { ?geoEntity gn:parentCountry  [ gn:name ?nameParentCountry ] }.
} group by ?geoEntity ?name ?latitude ?longitude ?altitude ?officialName ?nameParentCountry

geoEntity,name,latitude,longitude,altitude,officialName,alternateNames,nameParentCountry
https://sws.geonames.org/124544/,Māzandarān,36.25,52.33333,,Mazandaran,Māzandarān Province,Iran
https://sws.geonames.org/132892/,Gorgan,36.8427,54.44391,,,Gorgan,Iran
https://sws.geonames.org/130758/,Iran,32,53,,Iran,"Empire of Iran, Iran, Islamic Republic Of, Islamic Republic of Iran",


### 2.1 Find the geographic NEs of articles with descriptor "cacao"

Look for the articles that have the Agrovoc "cacao" concept (http://aims.fao.org/aos/agrovoc/c_7713) as a descriptor.

Then get the geographic named entities of those articles and retrieve their details from GeoNames.

In [139]:
SELECT distinct ?geoEntity ?name ?latitude ?longitude
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    []  a issa:GeographicAnnotation;
        schema:about ?article;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name         ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
} limit 10

geoEntity,name,latitude,longitude
https://sws.geonames.org/3017382/,France,46,2
https://sws.geonames.org/3469034/,Brazil,-10,-55
https://sws.geonames.org/2635167/,United Kingdom,54.75844,-2.69531
https://sws.geonames.org/3624060/,Costa Rica,10,-84
https://sws.geonames.org/3658394/,Ecuador,-1.25,-78.25
https://sws.geonames.org/2646057/,Ipswich,52.05917,1.15545
https://sws.geonames.org/2134431/,Vanuatu,-16,167
https://sws.geonames.org/4350359/,Capitol Heights,38.88511,-76.91581
https://sws.geonames.org/1733045/,Malaysia,2.5,112.5
https://sws.geonames.org/6252001/,United States,39.76,-98.5


### 2.2 Find the articles with descriptor "cacao" and their geographic NEs

Similar to the previous query but also return the articles.

In [141]:
SELECT distinct ?article ?articleVisu ?geoEntity ?name ?latitude ?longitude ?altitude
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
FROM <http://data-issa.cirad.fr/graph/thematic-descriptors>
WHERE {
    []  a issa:ThematicDescriptorAnnotation;
        oa:hasTarget ?article;
        oa:hasBody agrovoc:c_7713. # Concept "cacao"

    []  a issa:GeographicAnnotation;
        schema:about ?article;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name         ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
    
    ?article dct:identifier ?id.
    bind(iri(concat("http://issa.i3s.unice.fr/visu/?uri=", ?article)) as ?articleVisu)
} limit 10

article,articleVisu,geoEntity,name,latitude,longitude,altitude
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3017382/,France,46,2,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3469034/,Brazil,-10,-55,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/2635167/,United Kingdom,54.75844,-2.69531,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3624060/,Costa Rica,10,-84,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/3658394/,Ecuador,-1.25,-78.25,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/2646057/,Ipswich,52.05917,1.15545,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/2134431/,Vanuatu,-16,167,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/4350359/,Capitol Heights,38.88511,-76.91581,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/1733045/,Malaysia,2.5,112.5,
http://data-issa.cirad.fr/article/568597,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/568597,https://sws.geonames.org/6252001/,United States,39.76,-98.5,


### 3. Find the articles with geographic NEs within a certain distance of a given point

In [142]:
SELECT distinct ?article ?articleVisu
FROM <http://data-issa.cirad.fr/graph/articles>
FROM <http://geonames.org/graph>
FROM <http://data-issa.cirad.fr/graph/geographic-nes>
WHERE {
    []  a issa:GeographicAnnotation;
        schema:about ?article;
        oa:hasBody ?geoEntity.

    ?geoEntity
        gn:name   ?name;
        geo:lat   ?latitude;
        geo:long  ?longitude.
    
    FILTER (bif:st_within(bif:st_point(xsd:float(?longitude), xsd:float(?latitude)), bif:st_point (8, 10), 2000))
    
    ?article dct:identifier ?id.
    bind(iri(concat("http://issa.i3s.unice.fr/visu/?uri=", ?article)) as ?articleVisu)
} LIMIT 10

article,articleVisu
http://data-issa.cirad.fr/article/455935,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/455935
http://data-issa.cirad.fr/article/433785,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/433785
http://data-issa.cirad.fr/article/395431,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/395431
http://data-issa.cirad.fr/article/454957,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/454957
http://data-issa.cirad.fr/article/455868,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/455868
http://data-issa.cirad.fr/article/394049,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/394049
http://data-issa.cirad.fr/article/410537,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/410537
http://data-issa.cirad.fr/article/455078,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/455078
http://data-issa.cirad.fr/article/389182,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/389182
http://data-issa.cirad.fr/article/599920,http://issa.i3s.unice.fr/visu/?uri=http://data-issa.cirad.fr/article/599920
